In [43]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

from mne_connectivity import spectral_connectivity_epochs
from mne.datasets import sample
import seaborn as sns

from scipy.signal import welch
import yasa
import constants
import numpy as np

# importing random forest classifier from ensemble module
from sklearn.ensemble import RandomForestClassifier
# metrics are used to find accuracy or error
from sklearn import metrics  

# from sklearn model selection import Grid Search CV
from sklearn.model_selection import GridSearchCV

In [32]:
#1. Load all necessary data for training and testing ___________________________________________________________
#Folder for loading train-test indices, filtered TSFresh Feats, selected connectivity metric
folder = '../Toy_Data/Feature_Generation/Filtered_Features/' 

train_test_indices_dict = joblib.load(folder + 'train_test_indices.pkl') 

X_train_tsfresh = joblib.load(folder + 'X_train_tsfresh_features_filtered.pkl')
X_test_tsfresh = joblib.load(folder + 'X_test_tsfresh_features_filtered.pkl')

# First load the name of the selected metric into the variable selected_metric
with open(folder + 'selected_connectivity_metric.txt', "r") as f:
    selected_metric = f.read()

# Next Load the selected connectivity metric features from the connectivity metrics folder
connectivity_metrics_folder = '../Toy_Data/Feature_Generation/All_Features/'
X_connectivity = joblib.load(connectivity_metrics_folder + selected_metric)

X_connectivity_train = X_connectivity.iloc[train_test_indices_dict['train_idx']]
X_connectivity_test = X_connectivity.iloc[train_test_indices_dict['test_idx']]

# Finally load the y_labels and construct y_train and y_test
test_eeg_data =  np.load('../Toy_Data/test_data.npz')
y_labels = test_eeg_data['y']

y_train = y_labels[train_test_indices_dict['train_idx']]
y_test = y_labels[train_test_indices_dict['test_idx']]

#We now have : 
# X_train_tsfresh, X_test_tsfresh , X_connectivity_train, X_connectivity_test, y_train, y_test
# Reset indices
X_train_tsfresh.reset_index(drop = True, inplace = True), X_test_tsfresh.reset_index(drop = True, inplace = True) , X_connectivity_train.reset_index(drop = True, inplace = True), X_connectivity_test.reset_index(drop = True, inplace = True)

(None, None, None, None)

## Train the Model (Random Forest)

In [39]:
# X_train_tsfresh, X_test_tsfresh , X_connectivity_train, X_connectivity_test, y_train, y_test
X_train_full = pd.concat([X_train_tsfresh, X_connectivity_train], axis = 1)
X_test_full = pd.concat([X_test_tsfresh, X_connectivity_test], axis = 1)

### Train and Test a random forest with no tuning

In [59]:
# Train and test a random forest with no tuning:
clf = RandomForestClassifier( n_jobs = -1)
clf.fit( X_train_full , y_train )
y_pred = clf.predict(X_test_full)
acc = metrics.accuracy_score(y_test, y_pred)
acc

0.4166666666666667

### Train and Test a random forest with tuning

In [64]:
## Create the param grids for each model type
RF_param_grid = { 
    'n_estimators': [2,3, 10, 40,100,250],
    'max_features': [None, 'sqrt'],
    'max_depth' : [2,3,5,8, None],
    'criterion' :['gini',  'entropy'],
    'min_samples_split' : [2,3,4,5]}

RF_param_grid = { 
    'n_estimators': [2,3, 10] ,
    'max_features': [None, 'sqrt'],
    'max_depth' : [2,3],
    'criterion' :['entropy'],
    'min_samples_split' : [4]}

clf = GridSearchCV( RandomForestClassifier(), RF_param_grid , n_jobs = -1, refit = True, verbose = 1 )

clf.fit( X_train_full , y_train )

print(clf.best_params_)

y_pred = clf.predict(X_test_full)
acc = metrics.accuracy_score(y_test, y_pred)
acc

Fitting 5 folds for each of 12 candidates, totalling 60 fits
{'criterion': 'entropy', 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 10}


0.3333333333333333